In [1]:
import requests, re
from bs4 import BeautifulSoup
from IPython.display import HTML, display

import pandas as pd
 
# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer
# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer

# goes to page 247
# goes to page 562

# first page of thread:
# store data in lists --> pandas dataframe

# THREAD NAME
# <h3 class="post-title"><a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941">I can&#039;t live without my dad - anyone feel the same?</a></h3>

# THREAD ADDRESS
# page_link

# USERNAME
# <div class="field field-name-ds-user-picture field-type-ds field-label-hidden">
# <div class="field-items"><div class="field-item even">
# <a href="/about-cancer/cancer-chat/users/paulus">

# POST DATE
# <span class="post-created hidden-xs">22 Jun 2020 16:35</span>

# POST TEXT
# <div class="field field-name-field-harmony-text field-type-text-long field-label-hidden">

# POST IS A REPLY TO POST_ID
# <span class="post-is-reply-to">22 Jun 2020 21:04 in response to <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941" class="active">bras1548</a></span>

# POST ID
# <a name="post-1305941"></a>


#### FUNCTION: SCRAPE A THREAD PAGE

In [2]:
# FUNCTION: SCRAPE A THREAD PAGE
def page_scraper_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')
    
    # THREAD ADDRESS 
    thread_address_list = []
    thread_address = soup.find("link",{"rel":"canonical"})['href']
    thread_address = page_link
    #print(thread_address)

    # THREAD NAME
    thread_name_list = []
    thread_group = soup.find("h3", {"class": "post-title"})
    thread_name = thread_group.find("a").text
    #print(thread_name)

    # POST ID
    post_id_list = []
    post_id_group = soup.findAll("a", {"name": re.compile(r".*")})

    for post_id_g in post_id_group:
        if "post-" in post_id_g["name"]:
            post_id_list.append(post_id_g["name"])
            #print(post_id_g["name"])

    # POST USERNAME
    username_list = []
    user_group = soup.findAll("div", {"class": "field field-name-ds-user-picture field-type-ds field-label-hidden"})
    
    for user_g in user_group:    
        username = user_g.find("a")["href"]
        username = username.replace("/about-cancer/cancer-chat/users/", "")
        username_list.append(username)

        # populate THREAD ADDRESS, THREAD NAME lists
        thread_address_list.append(thread_address)
        thread_name_list.append(thread_name)
        #print(username)

    # POST DATE
    date_list = []
    date_group = soup.findAll("span", {"class": "post-created hidden-xs"})

    for date_g in date_group:
        date = date_g.text
        date_list.append(date)
        #print(date)

    # POST TEXT
    text_list = []
    text_group = soup.findAll("div", {"class": "field field-name-field-harmony-text field-type-text-long field-label-hidden"})

    for text_g in text_group:
        text_tag = text_g.find("div").find("div")
        text = text_tag.text
        text_list.append(text)
        #print(text_tag.text, "\n")

    # POST IS A REPLY TO
    reply_to_list = []
    reply_group = soup.findAll("span", {"class": "post-is-reply-to"})

    # add "N/A only to first posts in thread
    
    reply_group_len = len(reply_group)
    text_group_len = len(text_group)
    
    while reply_group_len < text_group_len:  # if CHANGED TO while
        reply_to_list.append("N/A")
        reply_group_len += 1
        
    for reply_g in reply_group:
        reply_to_id = reply_g.find("a")['href']
        reply_to_id = "https://www.cancerresearchuk.org" + reply_to_id
        reply_to_id = reply_to_id.replace(thread_address + "#", "")
        reply_to_list.append(reply_to_id)
        #print(reply_to_id)

    # COLUMN LISTS INTO A LIST OF ROWS
    new_rows = []

    for j in range(len(thread_name_list)):
        row_list = [thread_name_list[j], thread_address_list[j], username_list[j],
                    post_id_list[j], date_list[j], text_list[j], reply_to_list[j]]

        new_rows.append(row_list)
    
    return new_rows



#### FUNCTION: CHECK FOR "NEXT" PAGE IN THREAD

In [3]:
# FUNCTION: CHECK FOR "NEXT" PAGE IN THREAD

# <li class="cr-pagination__next">
# <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=1">Next</a></li>

def next_page_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')

    next_link = soup.find("li", {"class":"cr-pagination__next"})

    if next_link:
        next_link = next_link.find("a")['href']
        next_link = "https://www.cancerresearchuk.org/" + next_link
        #print(next_link)
        return next_link

    else:
        #print("last page")
        return None

print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road?page=1"))
print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"))



None
https://www.cancerresearchuk.org//about-cancer/cancer-chat/thread/the-end-of-the-road?page=1


In [4]:
# TEST PAGE SCRAPER FUNCTION

page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"
new_rows = page_scraper_cancer_research(page_link)


In [5]:
for row in new_rows:
    print(row[0])
    print(row[1])
    print(row[2])
    print(row[3])
    print(row[4])
    print(row[5])
    print(row[6], "\n")

The end of the road
https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road
goodhubby
post-1458971
19 Dec 2020 10:49
I always hoped I would never be writing this but my wife was given the news last week that her metastatic breast cancer was no longer treatable. Seven years ago she was diagnosed with stage 1b HER + tumour in her right breast which was surgically removed followed by radiotherapy and chemo, immunotherapy etc then at 4 years 10 months she was diagnosed with brain mets which were removed followed by radio therapy an chemo again, this year in April the scans showed liver mets and a small one in her lung. The hard part for me was to watch her look so beautiful again and then have to go through the same over again. The liver mets would not respond to any chemo of which 3 were tried and it was only when my wife begged the consultant to try another that she was offered it but we were told that it won't work due to the last few not working.  Her liver

#### FUNCTION: SCRAPE ONE THREAD

In [6]:
# FUNCTION: SCRAPE ONE THREAD

row_list = []
page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"

def thread_scraper_cancer_research(page_link):

    new_rows = page_scraper_cancer_research(page_link)
    next_link = next_page_cancer_research(page_link)
    
    #for row in new_rows:  # REMOVE THIS LATER
    #    print(row[0])
    #    print(row[1])
    #    print(row[2])
    #    print(row[3])
    #    print(row[4])
    #    print(row[5])
    #    print(row[6], "\n")
        
    
    if next_link is not None:
        new_rows.extend(thread_scraper_cancer_research(next_link))

    return new_rows

In [7]:
# TEST: SCRAPE ONE THREAD

page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"

new_rows = thread_scraper_cancer_research(page_link)
print(len(new_rows))

26


#### FUNCTION: GET THREAD LINKS FROM PAGE

In [8]:
# FUNCTION: GET THREAD LINKS FROM PAGE

forum_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"

def thread_link_scraper_cancer_research(forum_link):

    thread_links = []
    req = requests.get(forum_link)
    soup = BeautifulSoup(req.text, 'lxml')

    # <div class="tl-cell views-field views-field-nothing" >
    # <a href="/about-cancer/cancer-chat/thread/has-anyone-tried-bereavement-counselling">

    thread_link_group = soup.find_all("div", {"class":"tl-cell views-field views-field-nothing"})
    
    for thread_link_gr in thread_link_group:    
        new_thread_link = thread_link_gr.find('a')['href']
        thread_links.append("https://www.cancerresearchuk.org" + new_thread_link)
        #print("https://www.cancerresearchuk.org" + new_thread_link, "\n")
        
    return thread_links
    


#### FUNCTION: CHECK FOR "NEXT" PAGE IN FORUM

In [9]:
# FUNCTION: CHECK FOR "NEXT" PAGE IN FORUM

def next_forum_page_cancer_research(forum_link):
    
    req = requests.get(forum_link)
    soup = BeautifulSoup(req.text, 'lxml')

# <li class="next last"><a title="Go to next page" href="
    next_link = soup.find("li", {"class":"next last"})

    if next_link:
        next_link = next_link.find("a")['href']
        next_link = "https://www.cancerresearchuk.org" + next_link
        #print(next_link)
        return next_link

    else:
        #print("last page")
        return None

print(next_forum_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer"))
print(next_forum_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=562"))



https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=1
None


#### FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

In [10]:
# FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

def thread_links_cancer_research(forum_link):
    
    thread_links = []
    current_page = forum_link
    new_links = thread_link_scraper_cancer_research(current_page)
    thread_links.extend(new_links)
    
    next_link = next_forum_page_cancer_research(current_page)
    print(next_link)
    
    if next_link is not None:
        thread_links.extend(thread_links_cancer_research(next_link))
        
    return thread_links




In [11]:
# TEST FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

forum_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer"
thread_links = thread_links_cancer_research(forum_link)


https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=1
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=2
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=3
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=4
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=5
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=6
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=7
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=8
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=9
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=81
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=82
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=83
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=84
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=85
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=86
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=87
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=88
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=89
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-canc

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=160
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=161
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=162
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=163
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=164
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=165
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=166
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=167
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=168
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-a

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=239
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=240
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=241
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=242
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=243
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=244
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=245
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=246
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=247
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-a

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=318
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=319
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=320
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=321
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=322
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=323
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=324
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=325
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=326
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-a

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=397
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=398
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=399
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=400
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=401
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=402
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=403
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=404
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=405
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-a

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=476
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=477
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=478
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=479
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=480
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=481
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=482
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=483
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=484
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-a

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=555
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=556
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=557
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=558
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=559
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=560
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=561
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/living-with-cancer?page=562
None


In [13]:
# OPEN FILE WITH SAVED 'LIVING WITH CANCER' LINKS

thread_links = pd.read_csv(r'/Users/Sofia/Desktop/cancer_research_living_cancer_links.csv', delimiter="\t")
thread_links = thread_links["0"].values.tolist()
print(len(thread_links))


#test_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

#for i in test_list[0:3]:
#    print(i)
    
#print("\n")

#for i in test_list[3:6]:
#    print(i)
    
#print("\n")

#for i in test_list[6:]:
#    print(i)

5623


In [108]:
# DEBUGGING PROBLEM PAGE

bug_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/confused-30"
#bug_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/seeing-signs-from-deceased-loved-ones?page=7"

bug_rows = thread_scraper_cancer_research(bug_link)
#bug_rows = page_scraper_cancer_research(bug_link)

In [110]:
#for line in bug_rows:
#    print(line[0])
#    print(line[1])
#    print(line[2])
#    print(line[3])
#    print(line[4])
#    print(line[5])
#    print(line[6], "\n")

In [45]:
# RUN 'SCRAPE THREAD' ON THREAD_LINKS

#living_cancer_data = [] # DO NOT RESET !!

count = 5001

for thread in thread_links[5000:]:
    
    print(count, thread)

    new_rows = thread_scraper_cancer_research(thread)    
    living_cancer_data.extend(new_rows)
    
    count += 1



5001 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/there-is-hope-0
5002 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/langerhans-cell-sarcoma
5003 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/kidney-cancer-survivors
5004 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/worrying-swelling-on-ribs-3-yrs-after-kidney-cancer
5005 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/symptoms-of-a-brain-tumor-am-i-worrying-over-nothing
5006 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/grandad-dying-and-im-on-chemo
5007 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/free-flap-fitted-and-neck-dissection-advice-please
5008 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/very-tired-dad-any-tips
5009 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/hi-all
5010 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-do-you-cope-1
5

5086 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/anyone-need-to-chat
5087 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/further-tests
5088 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/surgery-yes-or-no
5089 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/post-op-infections
5090 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/hysterectomy-due-to-ovarian-cancer
5091 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/post-op
5092 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/bowel-cancer-5
5093 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/why-do-i-get-soooooo-flushed-in-the-face-and-neck-but-have-a-normal-temp
5094 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/hi-10
5095 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/grave
5096 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/yes-or-no-to-che

5173 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/laryngectomy
5174 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/small-cell-lung-cancer-and-folic-acid-help
5175 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/scarey
5176 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-have-had-enough-gimme-my-life-back
5177 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/still-in-severe-pain-has-anyone-else-experienced-this
5178 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/prostate-cancer-12
5179 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/another-stupid-post-by-sj-the-selfish-fraud
5180 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/gall-bladder-liver-cancer
5181 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/friend-with-breast-cancer
5182 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/after-effects-of-chemotherapy
5

5257 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/oxydhq-and-zeolite
5258 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/is-there-a-cure
5259 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/who-can-help-me
5260 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-have-another-idea-to-raise-money-for-my-idea-with-cancer-research
5261 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/physical-and-mental
5262 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-too-would-like-to-talk-about-breast-cancer-0
5263 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/moles
5264 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/breast-cancer-5
5265 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/if-you-would-like-to-chat-here-is-someone
5266 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-challenges-since-having-a-brain-tumour-diagnose

5343 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/medical
5344 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/cinnamon-trust
5345 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/not-again
5346 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/have-had-lump-removed
5347 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/online-self-diagnosis/worrying/research-and-the-waiting-game
5348 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/to-everyone
5349 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/discharged-to-early
5350 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dont-no-what-to-do
5351 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-other-people-see-you
5352 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/bowel-cancer-0
5353 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/does-anyone-who-is-in-r

5426 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/nhl
5427 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-do-i-look
5428 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/proper-places-to-get-advice
5429 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-found-a-lump
5430 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/loss-of-mobility
5431 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/all
5432 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/an-alleged-misdiagnosis
5433 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/right-2-life-for-adopted-children-adults-2-find-a-genetic-key
5434 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-need-help
5435 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/thanks
5436 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/just-had-bad-news
5437 https://www.cancerre

5510 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/um-advice-please
5511 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/thankyou
5512 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/add-a-profile-photo-to-cancer-chat
5513 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/caring-sharing
5514 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/recurring-cancers
5515 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/have-you-taken-part-in-the-tango-or-azzure-trials-for-breast-cancer-0
5516 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/late-birthday-remembrance
5517 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/where-can-i-live-chat
5518 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/accountability
5519 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/new-important-petition-childhood-young-adult-leukaemia
5520 https://www.c

5594 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/hi-0
5595 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/help
5596 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/advice-needed
5597 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-wife
5598 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/what-is-wrong
5599 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/30at30-challenges
5600 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/salford-relay-for-life-2009
5601 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/post-operation-wound-healing
5602 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/relay-for-life
5603 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/why-does-diagnosis-take-so-long
5604 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/feel-good-films
5605 https://www.cancerresearchuk.org/about-ca

In [47]:
# SAVE PRELIMINARY DATA

column_names = ['Thread_name', 'Thread_address', 'Username', "Post_id",
                                  "Date", "Post_text", "Reply_to_id"]

cancer_research_living_cancer_prelim = pd.DataFrame(living_cancer_data, columns = column_names)  

cancer_research_living_cancer_prelim.to_csv(r'/Users/Sofia/Desktop/cancer_research_living_cancer.csv', sep ='\t', index = False, header=True)




In [46]:
print(len(living_cancer_data))

# print(len(thread_links))
    
#for line in living_cancer_data[0:300]:
#    print(line[0])
#    print(line[1])
#    print(line[2])
#    print(line[3])
#    print(line[4])
#    print(line[5])
#    print(line[6], "\n")


48646


In [23]:
# SAVE 'LIVING WITH CANCER' FORUM LINKS TO FILE

#print(len(thread_links))

#for thread in thread_links:
#    print(thread)

cancer_research_living_cancer_links = pd.DataFrame(thread_links)
cancer_research_living_cancer_links.to_csv(r'/Users/Sofia/Desktop/cancer_research_living_cancer_links.csv', sep ='\t', index = False, header=True)



In [137]:
# GLOBAL database for 
# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss

column_names = ['Thread_name', 'Thread_address', 'Username', "Post_id",
                                  "Date", "Post_text", "Reply_to_id"]

cancer_research_coping_w_loss = pd.DataFrame(new_rows, columns = column_names)  
print (cancer_research_coping_w_loss)

cancer_research_coping_w_loss.to_csv(r'/Users/Sofia/Desktop/cancer_research_coping_loss_test.csv', sep ='\t', index = False, header=True)


                                          Thread_name  \
0   I can't live without my dad - anyone feel the ...   
1   I can't live without my dad - anyone feel the ...   
2   I can't live without my dad - anyone feel the ...   
3   I can't live without my dad - anyone feel the ...   
4   I can't live without my dad - anyone feel the ...   
5   I can't live without my dad - anyone feel the ...   
6   I can't live without my dad - anyone feel the ...   
7   I can't live without my dad - anyone feel the ...   
8   I can't live without my dad - anyone feel the ...   
9   I can't live without my dad - anyone feel the ...   
10  I can't live without my dad - anyone feel the ...   
11  I can't live without my dad - anyone feel the ...   
12  I can't live without my dad - anyone feel the ...   
13  I can't live without my dad - anyone feel the ...   
14  I can't live without my dad - anyone feel the ...   
15  I can't live without my dad - anyone feel the ...   
16  I can't live without my dad